In [2]:
!pip install "package" --user

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [7 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Diametrix\AppData\Local\Temp\pip-install-jv9d93nz\package_091a0ac4d5cf4347aae8463f028dc294\setup.py", line 4
      print """
            ^
  SyntaxError: Missing parentheses in call to 'print'. Did you mean print(""")?
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install -U osmnx 

  Using cached osmnx-1.3.0-py3-none-any.whl (93 kB)
  Using cached geopandas-0.12.2-py3-none-any.whl (1.1 MB)
  Attempting uninstall: geopandas
    Found existing installation: geopandas 0.11.1
    Uninstalling geopandas-0.11.1:
      Successfully uninstalled geopandas-0.11.1



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import pickle
from shapely import LineString,MultiLineString
import numpy as np
import difflib

In [276]:
path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm"

In [144]:
iris_data=pd.read_excel('Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - IRIS\\Geodata_IRIS_essentiels.xlsx')

In [145]:
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp')

In [25]:
regions=["Auvergne-Rhône-Alpes",
    "Bourgogne-Franche-Comté",
    "Bretagne",
    "Centre-Val de Loire",
    "Corse",
    "Grand Est",
    'Hauts-de-France',
    "Ile-de-France",
    "Normandie",
    "Nouvelle-Aquitaine",
    "Occitanie",
    'Pays de la Loire',
    "Provence-Alpes-Côte d'Azur"]


In [296]:
lyon_metropole=["Lyon",
"Albigny-sur-Saône",
"Bron",
"Cailloux-sur-Fontaines",
"Caluire-et-Cuire",
"Champagne-au-Mont-d'Or",
"Charbonnières-les-Bains",
"Charly",
"Chassieu",
"Collonges-au-Mont-d'Or",
"Corbas",
"Couzon-au-Mont-d'Or",
"Craponne",
"Curis-au-Mont-d'Or",
"Dardilly",
"Décines-Charpieu",
"Écully",
"Feyzin",
"Fleurieu-sur-Saône",
"Fontaines-Saint-Martin",
"Fontaines-sur-Saône",
"Francheville",
"Genay",
"Givors",
"Grigny",
"Irigny",
"Jonage",
"Limonest",
"Lissieu",
"Marcy-l'Étoile",
"Meyzieu",
"Mions",
"Montanay",
"La Mulatière",
"Neuville-sur-Saône",
"Oullins",
"Pierre-Bénite",
"Poleymieux-au-Mont-d'Or",
"Quincieux",
"Rillieux-la-Pape",
"Rochetaillée-sur-Saône",
"Saint-Cyr-au-Mont-d'Or",
"Saint-Didier-au-Mont-d'Or",
"Sainte-Foy-lès-Lyon",
"Saint-Fons",
"Saint-Genis-Laval",
"Saint-Genis-les-Ollières",
"Saint-Germain-au-Mont-d'Or",
"Saint-Priest",
"Saint-Romain-au-Mont-d'Or",
"Sathonay-Camp",
"Sathonay-Village",
"Solaize",
"Tassin-la-Demi-Lune",
"La Tour-de-Salvagny",
"Vaulx-en-Velin",
"Vénissieux",
"Vernaison",
"Villeurbanne"]

In [26]:
list_departement=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Liste-Excel-des-departements-francais.xlsx")

In [27]:
list_departement=list_departement[list_departement["Région"].isin(regions)]

# Batiments

In [314]:
def get_buildings(place_name,region):
    tags = {"building":True }
    
    if place_name.lower()!="rhône": 
        try:
            buildings = ox.geometries_from_place(place_name+", France", tags)
        except:
            buildings = ox.geometries_from_place(place_name, tags)
    else:
        buildings = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
        
    selected_columns=["geometry" ,"building", "shop" , "building:levels","amenity","name"]
    buildings=buildings[selected_columns]
    pickle.dump(buildings,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\buildings.pkl",'wb'))   
    return buildings

# amenity (hopital , école ...)

In [306]:
def get_amenity(place_name,region):
    tags = {"amenity":True }
    if place_name.lower()!="rhône": 
        try:
            amenity = ox.geometries_from_place(place_name+", France", tags)
        except:
            amenity = ox.geometries_from_place(place_name, tags)
    else:
        amenity = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
    selected_columns=["geometry" ,"building", "shop" , "building:levels","amenity","name"]
    amenity=amenity[selected_columns]
    
    pickle.dump(amenity,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\amenity.pkl","wb"))

# Natural

In [307]:
def get_natural (place_name,region):
    tags = {"natural":True }   
    if place_name.lower()!="rhône": 
        try:
            natural = ox.geometries_from_place(place_name+", France", tags)
        except:
            natural = ox.geometries_from_place(place_name, tags)
    else:
        natural = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
        
    natural=natural[["geometry",'natural','name']]
    pickle.dump(natural,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\natural.pkl","wb"))

# Landuse

In [308]:
def get_landuse(place_name,region):
    tags = {"landuse":True } 
    if place_name.lower()!="rhône": 
        try:
            landuse = ox.geometries_from_place(place_name+", France", tags)
        except:
            landuse = ox.geometries_from_place(place_name, tags)
    else:
        landuse = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
    landuse=landuse[["geometry",'landuse','name']]
    pickle.dump(landuse,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\landuse.pkl","wb"))

# Shops (restaurant ,Vêtements...)

In [309]:
def get_shops (place_name,region):
    tags = {"shop":True }   
    if place_name.lower()!="rhône": 
        try:
            shops = ox.geometries_from_place(place_name+", France", tags)
        except:
            shops = ox.geometries_from_place(place_name, tags)
    else:
        shops = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
        
    selected_columns=["geometry" ,"building", "shop" ,"amenity","name", ]
    shops=shops[selected_columns]
    pickle.dump(shops,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\shops.pkl","wb"))

# Transport

In [310]:
def get_transport(place_name,region):
    tags = {"railway":["station",'tram_stop'],"station":"subway"}
    if place_name.lower()!="rhône": 
        try:
            public_transports = ox.geometries_from_place(place_name+", France", tags=tags)
        except:
            public_transports = ox.geometries_from_place(place_name, tags=tags)
    else:
        public_transports = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
        
    selected_columns=['geometry' ,'public_transport','name']
    public_transports=public_transports[selected_columns]
    pickle.dump(public_transports,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\public_transports.pkl","wb"))

# Rues

In [311]:

def get_highway(place_name,region):
    tags = {"highway":True }
    
    if place_name.lower()!="rhône":  
        try:
            highways = ox.geometries_from_place(place_name+", France", tags)
        except:
            highways = ox.geometries_from_place(place_name, tags)
    else:

        highways = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
    selected_columns=['geometry' ,'highway','name']
    highways=highways[selected_columns]
    pickle.dump(highways,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\highways.pkl","wb"))
    return highways

# Tourism

In [312]:
def get_tourism(place_name,region):
    tags = {"tourism":True }   
    if place_name.lower()!="rhône": 
        try:
            tourism = ox.geometries_from_place(place_name+", France", tags)
        except:
            tourism = ox.geometries_from_place(place_name, tags)
    else:
        tourism = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
    selected_columns=["name", "geometry","tourism","stars"]
    tourism=tourism[selected_columns]
    pickle.dump(tourism,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\tourism.pkl","wb"))

# Loisir

In [313]:
def get_leisure(place_name,region):
    tags = {"leisure":True }
    if place_name.lower()!="rhône": 
        try:
            leisure = ox.geometries_from_place(place_name+", France", tags)
        except:
            leisure = ox.geometries_from_place(place_name, tags)
    else:
        leisure = ox.geometries_from_place([place_name,"métropole de Lyon"], tags)
        
        
    selected_columns=["name", "geometry","leisure"]
    leisure=leisure[selected_columns]
    pickle.dump(leisure,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\leisure.pkl","wb"))

# For Loop

In [161]:
#creation des dossiers
from pathlib import Path

for i,row in list_departement.iterrows():

        Path(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+"\\"+row["Département"].lower()+"\\raw data").mkdir(parents=True, exist_ok=True)
        Path(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+"\\"+row["Département"].lower()+"\\processed data").mkdir(parents=True, exist_ok=True)
        Path(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+"\\"+row["Département"].lower()+"\\association").mkdir(parents=True, exist_ok=True)


In [315]:
for i,row in list_departement.iterrows():
    place_name=row['Département']
  
    if place_name.lower()=="rhône":
        bat=get_buildings(place_name,row["Région"])
        get_amenity(place_name,row["Région"])
        get_natural(place_name,row["Région"])
        get_landuse(place_name,row["Région"])
        get_shops(place_name,row["Région"])
        get_tourism(place_name,row["Région"])
        get_highway(place_name,row["Région"])
        get_transport(place_name,row["Région"])
        get_leisure(place_name,row["Région"])


        

C:\Users\Diametrix\AppData\Local\Temp\ipykernel_24408\3480682846.py:14: ResourceWarning: unclosed file <_io.BufferedWriter name="Y:\\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\\osm\\auvergne-rhône-alpes\\rhône\\raw data\\buildings.pkl">
  pickle.dump(buildings,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\buildings.pkl",'wb'))
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_24408\3579390011.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name="Y:\\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\\osm\\auvergne-rhône-alpes\\rhône\\raw data\\amenity.pkl">
  pickle.dump(amenity,open(path+"\\"+region.lower()+"\\"+place_name.lower()+"\\raw data\\amenity.pkl","wb"))
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_24408\4119075942.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name="Y:\\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\\osm\\auvergne-r

In [317]:
import os
index=[]
for i,f in enumerate(list_departement.apply(lambda x:x["Région"].lower()+"\\"+x['Département'].lower()+"\\raw data",axis=1).values):
    if len(os.listdir(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+f)) !=9:
        index.append(i)


In [318]:
err_dep=list_departement.iloc[index]
err_dep

,N°,Département,Chef-lieu,Région
65,65,Hautes-Pyrénées,Tarbes,Occitanie
66,66,Pyrénées-Orientales,Perpignan,Occitanie
67,67,Bas-Rhin,Strasbourg,Grand Est
68,68,Haut-Rhin,Colmar,Grand Est
74,74,Haute-Savoie,Annecy,Auvergne-Rhône-Alpes
90,90,Territoire-de-Belfort,Belfort,Bourgogne-Franche-Comté


# Traitement des données

In [31]:
import pickle 
import pandas as pd
import numpy as np

In [320]:
#logs and stats
logs={}

important_columns=["unclassified", "residential","service","path","unclassified", "pedestrian","primary" ,"secondary" ,"tertiary","living_street" ,"trunk" ,"motorway" ]
for i ,row in list_departement.iterrows():
    try:
        place_name=row["Département"].lower()
        path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+r"\\"+row["Département"].lower()
        if place_name=="rhône":
            #Batiments
            data=pickle.load(open(path+r"\\raw data\buildings.pkl","rb")).to_crs("3857")
            n=len(data)
            data=data[data.area>=30]
            pickle.dump(data,open(path+r"\\processed data\buildings.pkl","wb"))
            #logs
            logs[place_name]={"batiment":{

                "types":data['building'].value_counts(),
                "building:levels":data['building:levels'].isna().sum(),
                "size_raw":n,
                "size":len(data),
                "type_yes":len(data[data['building']=="yes"])}
            }




            #Rues
            highways=pickle.load(open(path+r"\\raw data\highways.pkl","rb")).to_crs("3857")
            n=len(highways)
            highways=highways[highways['highway'].isin(important_columns)]
            highways=highways[~highways['name'].isna()]
            highways=highways[highways['geometry'].apply (lambda x:isinstance(x,(LineString,MultiLineString)))]
            highways=highways[ (highways['highway']!="service")]
            highways=highways[highways.length>=50]
            pickle.dump(highways,open(path+r"\\processed data\highways.pkl","wb"))
            #logs
            logs[place_name].update({"highways":{

                "types":highways['highway'].value_counts(),
                "size_raw":n,
                "size":len(highways),
            }})

            #Tourisme
            tourism=pickle.load(open(path+r"\\raw data\tourism.pkl","rb")).to_crs("3857")
            n=len(tourism)
            tourism=tourism[tourism['tourism']!="information"]
            pickle.dump(tourism,open(path+r"\\processed data\tourism.pkl","wb"))
            #logs
            logs[place_name].update({"tourism":{

                "types":tourism['tourism'].value_counts(),
                "size_raw":n,
                "size":len(tourism),
            }})

            #Amenity
            amenity=pickle.load(open(path+r"\\raw data\amenity.pkl","rb")).to_crs("3857")
            n=len(amenity)
            leisure=pickle.load(open(path+r"\\raw data\leisure.pkl","rb")).to_crs("3857")

            amenity=amenity.drop(np.intersect1d(np.array(tourism.index),np.array(amenity.index)))
            amenity=amenity.drop(np.intersect1d(np.array(leisure.index),np.array(amenity.index)))
            pickle.dump(amenity,open(path+r"\\processed data\amenity.pkl","wb"))
            #logs
            logs[place_name].update({"amenity":{

                "types":amenity['amenity'].value_counts(),
                "size_raw":n,
                "size":len(amenity),
            }})
            logs[place_name].update({"leisure":{

                "types":leisure['leisure'].value_counts(),
                "size":len(amenity),
            }})


    except:
        print(place_name)



C:\Users\Diametrix\AppData\Local\Temp\ipykernel_24408\513618213.py:11: ResourceWarning: unclosed file <_io.BufferedReader name="Y:\\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\\osm\\\\auvergne-rhône-alpes\\\\rhône\\\\raw data\\buildings.pkl">
  data=pickle.load(open(path+r"\\raw data\buildings.pkl","rb")).to_crs("3857")
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_24408\513618213.py:14: ResourceWarning: unclosed file <_io.BufferedWriter name="Y:\\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\\osm\\\\auvergne-rhône-alpes\\\\rhône\\\\processed data\\buildings.pkl">
  pickle.dump(data,open(path+r"\\processed data\buildings.pkl","wb"))
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_24408\513618213.py:29: ResourceWarning: unclosed file <_io.BufferedReader name="Y:\\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\\osm\\\\auvergne-rhône-alpes\\\\rhône\\\\raw data\\highways.pkl">
  high

In [33]:
pickle.dump(logs,open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\\logs.pkl","wb"))

In [324]:
aa=ox.geometries_from_place(["métropole de Toulouse"], {"building":True})

KeyboardInterrupt: 